This is the code for loading the batters' data from the lahman database that have data for all five years I am using (2014-2017 for the statistic and 2018 for evaluation).

### IMPORTANT NOTE
This code requires data thats way too big to be included in this repo so you'd have to do some serious downloading if you want to test the code on your own. I have however included the file that this code outputs.

In [77]:
import pandas as pd
import numpy as np

batting = pd.read_csv("data/Batting.csv")
players = pd.read_csv("data/People.csv")

batter_data = pd.merge(batting, players[["playerID", "nameLast", "nameFirst"]], on="playerID")

batter_data = batter_data.loc[batter_data["yearID"]>=2014]

players = set(list(batter_data["playerID"]))

tot = 0
for play in players: # Remove players that don't appear in all years
    for ii in range(2014,2019):
        if len(batter_data.loc[(batter_data["playerID"] == play) & (batter_data["yearID"] == ii)]) == 0:
            batter_data = batter_data[batter_data.playerID != play]
            break

at_bats = batter_data[["playerID", "AB"]].copy()

for ind,row in at_bats.groupby("playerID").sum().iterrows(): # Get only the players that have a total of at least 200 at bats
    if row["AB"] < 200:
        batter_data = batter_data[batter_data.playerID != ind]

players = set(batter_data["playerID"])



batter_data = batter_data.drop(columns=["teamID", "lgID", "stint"]) # I don't need these


batter_data["pyearID"] = batter_data["yearID"].apply(str) + batter_data["playerID"] # combined column for grouping

ag_key = {"playerID": "first", "yearID": "first", 'nameLast': "first", 'nameFirst': "first",'G':np.sum, 'AB':np.sum, 'R':np.sum, 'H':np.sum, '2B':np.sum, '3B':np.sum, 'HR':np.sum, 'RBI':np.sum, 'SB':np.sum, 'CS':np.sum, 'BB': np.sum, 'SO':np.sum, 'IBB':np.sum, 'HBP':np.sum, 'SH':np.sum, 'SF':np.sum, 'GIDP':np.sum}
# this is a custome aggreation so that players who played on more than one team in a year only have one entry in that year
# with their combine stats
batter_data = batter_data.groupby("pyearID").agg(ag_key).set_index("playerID") 

batter_data = batter_data.reset_index()

master = pd.read_csv("data/idkey.csv")

batter_data = pd.merge(batter_data, master, on="playerID")

Statcast stats
### Difficult pitches hit: DPH%
curve, split, screw, and knuckle balls hit 

### Fast balls hit: FBH%
percentage of balls over 95mph hit

### Ball call ability BCA
percentage of balls pitched outside of strike zone and correctly not swung at

I found the best way to store these stats is similar to how other stats are stored in the Lahman database in that I am storing the ones that qualify as a hit or a ball and then counting the total so that these two stats can be used to get the 

In [136]:
# I had to hand modify the 2017 and 2018 files as they were raising strange type assumption errors with all columns present
years = {2014: ["fourteen1.csv", "fourteen2.csv", "fourteen3.csv"], 2015: ["fifteen1.csv", "fifteen2.csv"],2016: ["sixteen1.csv", "sixteen2.csv", "sixteen3.csv"],
    2017: ["sev1.csv","sev2.csv","sev3.csv"], 2018:["eig1.csv", "eig2.csv", "eig3.csv"]}
for x in years:
    for ii in range(len(years[x])):
        years[x][ii] = "data/" + years[x][ii]

SyntaxError: invalid syntax (<ipython-input-136-3671bd2f2f37>, line 3)

In [95]:
batter_data["DPHh"] = 0 # intialize the statcast based stats
batter_data["DPHt"] = 0
batter_data["BCAh"] = 0
batter_data["BCAt"] = 0
batter_data["FBHh"] = 0
batter_data["FBHt"] = 0

In [96]:
pitches = ["CU", "FS", "KN", "SC", "KC"] # The difficult pitches
soz = ["11", "12", "13","14"] # These are the zones outside the strike box

In [97]:
players = set(batter_data["mlb_id"])

In [102]:
# functions for calcuating each stat
def dph(df, playerID):
    tot = df.loc[(df["batter"] == playerID) & df["pitch_type"].isin(pitches)]
    hits = len(tot.loc[(tot["type"] == "X")])
    return hits, len(tot)
def fbh(df, playerID):
    tot = df.loc[(df["batter"] == playerID) & (df["release_speed"] > 95)]
    hits = len(tot.loc[(tot["type"] == "X")])
    return hits, len(tot)
def bca(df, playerID):
    alls = df.loc[(df["batter"] == playerID) & df["zone"].isin(soz)]
    balls = len(alls.loc[(alls["type"] == "B")])
    return balls, len(alls)

In [133]:
for year in years: # query all the statcast files for all the players
    print(year) # this took way too long to run
    ii = 0
    for ID in players:
        ii += 1
        if ii % 40 == 0:
            print(ii)
        dhi = 0
        dto = 0
        fhi = 0
        fto = 0
        bhi = 0
        bto = 0
        for file in years[year]:
            df = pd.read_csv(file, dtype={"batter": int, "zone": float, "type": str, "release_speed": float, "pitch_type": str})
            dh, dt = dph(df, ID)
            fh, ft = fbh(df, ID)
            bh, bt = bca(df, ID)
            dhi += dh
            dto += dt
            fhi += fh
            fto += ft
            bhi += bh
            bto += bt
        batter_data.loc[(batter_data["yearID"] == year) & (batter_data["mlb_id"] == ID), "DPHh"] += dhi
        batter_data.loc[(batter_data["yearID"] == year) & (batter_data["mlb_id"] == ID), "DPHt"] += dt
        batter_data.loc[(batter_data["yearID"] == year) & (batter_data["mlb_id"] == ID), "BCAh"] += bh
        batter_data.loc[(batter_data["yearID"] == year) & (batter_data["mlb_id"] == ID), "BCAt"] += bt
        batter_data.loc[(batter_data["yearID"] == year) & (batter_data["mlb_id"] == ID), "FBHh"] += fh
        batter_data.loc[(batter_data["yearID"] == year) & (batter_data["mlb_id"] == ID), "FBHt"] += ft
            

2018
40
80
120
160
200
240
280


In [135]:
batter_data.to_csv("batter_data.csv") # save the gathered data so that the above code may never be ran again 